<a href="https://colab.research.google.com/github/syma-afsha/Complete_Reinforcement_Learning/blob/main/MDP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Define a Markov Decision Process (MDP) with
𝑁 states (blocks) and two possible actions for moving between states:


"walk": Moves the state forward by 1 (deterministic, costs
−
1
−1).

"tram": Attempts to double the state (50% chance of success, 50% chance of failure, costs
−
2
−2).

The goal is to find the optimal policy and state values to minimize the total penalty (maximize rewards) starting from state 1 and ending at the terminal state
𝑁.

In [20]:
class MDP(object):
    # Initialize the MDP with N number of states (blocks)
    def __init__(self, N):
        self.N = N  # Number of blocks (states)

    # Define the start state (always starts at state 1)
    def StartState(self):
        return 1

    # Check if the given state is the terminal state (goal state)
    def isEnd(self, state):
        return state == self.N

    # Define possible actions for a given state
    def Actions(self, state):
        result = []
        # If walking to the next state is valid, add "walk" action
        if state + 1 <= self.N:
            result.append("walk")
        # If using the tram to double the state is valid, add "tram" action
        if state * 2 <= self.N:
            result.append("tram")
        return result

    # Define transition probabilities, rewards, and next states for each action
    def TransitionProb(self, state, action):
        result = []
        if action == "walk":
            # "walk" always transitions to state+1 with probability 1 and a reward of -1
            result.append((state + 1, 1, -1))
        if action == "tram":  # action == "tram"
            # "tram" has a 50% chance to double the state and a 50% chance to fail
            result.append((state * 2, 0.5, -2))  # Successful tram
            result.append((state, 0.5, -2))  # Failed tram (stay in the same state)
        return result

    # Define the discount factor (γ), which reduces the value of future rewards
    def discount(self):
        return 0.8

    # Return a list of all possible states (1 to N)
    def states(self):
        return range(1, self.N + 1)

# Perform Value Iteration to compute optimal state values and policy
def ValueIteration(mdp, epsilon=1e-10):
    # Initialize state values (V) to 0 for all states
    V = {state: 0 for state in mdp.states()}

    # Function to compute Q-value for a given state and action
    def Q(state, action):
        # Q(s, a) = Σ (probability * (reward + γ * V(new state)))
        return sum(prob * (reward + mdp.discount() * V[newState])
                   for newState, prob, reward in mdp.TransitionProb(state, action))

    iteration = 0
    while True:
        delta = 0  # Track the maximum change in state values
        newV = V

        for state in mdp.states():
            if mdp.isEnd(state):
                # Terminal state has a value of 0
                newV[state] = 0
            else:
                # Update state value as the maximum Q-value across all actions
                newV[state] = max(Q(state, action) for action in mdp.Actions(state))

            # Update delta to track the maximum value change
            delta = max(delta, abs(newV[state] - V[state]))

        V = newV  # Update the value function
        iteration += 1

        # Check for convergence
        if delta < epsilon:
            break

    # Compute the optimal policy based on the final value function
    policy = {}
    for state in mdp.states():
        if mdp.isEnd(state):
            # Terminal state has no action
            policy[state] = None
        else:
            # Optimal action is the one with the highest Q-value
            policy[state] = max((Q(state, action), action) for action in mdp.Actions(state))[1]

    # Print the final results
    print(f"Converged in {iteration} iterations.")
    for state in mdp.states():
        print(f"State: {state}, Optimal Policy: {policy[state]}, Optimal Value: {V[state]:.2f}")

# Create an MDP with 10 states (blocks)
mdp = MDP(N=10)
ValueIteration(mdp)


Converged in 1 iterations.
State: 1, Optimal Policy: walk, Optimal Value: -1.00
State: 2, Optimal Policy: walk, Optimal Value: -1.00
State: 3, Optimal Policy: walk, Optimal Value: -1.00
State: 4, Optimal Policy: walk, Optimal Value: -1.00
State: 5, Optimal Policy: walk, Optimal Value: -1.00
State: 6, Optimal Policy: walk, Optimal Value: -1.00
State: 7, Optimal Policy: walk, Optimal Value: -1.00
State: 8, Optimal Policy: walk, Optimal Value: -1.00
State: 9, Optimal Policy: walk, Optimal Value: -1.00
State: 10, Optimal Policy: None, Optimal Value: 0.00
